In [66]:
import typing
import copy
import pdb
from IPython.display import clear_output
import numpy as np
import time
from colorama import Fore
import sys
import cProfile

In [53]:
#global variables that are never mutated
pieces = ["Pawn","Rook","Knight","Bishop","Queen","King"]
list_letters = ['a','b','c','d','e','f','g','h']
list_numbers = list(map(str,list((range(1,9)))))
legal_outputs = ["pawn_move","pawn_move,en-passant","king_move",'king_move,castling,kingside','king_move,castling,queenside','en-passant','']
en_passant = ["c7c5","a2a3","c5c4","b2b4","c4b3"]

In [3]:
#Creates a new board
def create_board(): 
    board = np.array([[[0,0,0,0,0,0] for j in range(8)] for i in range(8)])
    for i in range(8):
        board[1,i,0] = 1
        board[6,i,0] = -1
        
    board[0,4,5] = 1
    board[7,4,5] =  -1

    board[0,1,2] = 1
    board[7,1,2] = -1

    board[0,6,2] = 1
    board[7,6,2] = -1

    board[0,2,3] = 1
    board[7,2,3] = -1

    board[0,5,3] = 1 
    board[7,5,3] = -1

    board[0,3,4] = 1
    board[7,3,4] = -1

    board[0,4,5] = 1
    board[7,4,5] = -1
    
    board[0,0,1] = 1
    board[7,0,1] = -1 
    
    board[0,7,1] = 1
    board[7,7,1] = -1
    return [board]

def create_rights():
    rights = np.array([[0 for j in range(8)] for i in range(8)])
    rights[0,4] = 1
    rights[7,4] = -1
    
    rights[0,0] = 1
    rights[7,0] = -1
    rights[0,7] = 1
    rights[7,7] = -1
    return [rights]

def create_king_positions():
    king_positions = [[0,4],[7,4]]
    return [king_positions]


def pp_board(board):
    for i in range(8):
        print(Fore.BLACK + list_numbers[7-i],end="\t")
        for j in range(8):
            one_hot_piece = board[i,j]
            player = sum(one_hot_piece)
            if player == 0:
                print(Fore.BLACK + "_",end="\t")
            else: 
                index = next(i for i,v in enumerate(one_hot_piece) if v!=0)
                piece = pieces[index]
                if player== 1:
                    print(Fore.BLUE + piece,end="\t")
                else:
                    print(Fore.BLACK + piece,end="\t")
        print("\n")
    print("",end="\t")
    for i in list_letters:
        print(Fore.BLACK + i,end="\t")
    sys.stdout.flush()

In [4]:
#defining some directions
north = [-1,0]
south = [1,0]
east =  [0,1] 
west= [0,-1]
northwest= [-1,-1] 
northeast= [-1,1] 
southwest = [1,-1] 
southeast= [1,1] 
cross_dir = [north, south, east,west]
lateral_dir = [northeast,northwest,southeast,southwest]
all_dir = cross_dir + lateral_dir
all_dir.extend(lateral_dir)
NNW = np.add(np.add(north,north),west)
NWW = np.add(np.add(north,west),west)
NNE = np.add(np.add(north,north),east)
NEE = np.add(np.add(north,east),east)
SSW = np.add(np.add(south,south),west)
SWW = np.add(np.add(south,west),west)
SSE = np.add(np.add(south,south),east)
SEE = np.add(np.add(south,east),east)
knight_dir = [NNW,NWW,NNE,NEE,SSW,SWW,SSE,SEE]
knight_dir = [i.tolist() for i in knight_dir ]
direction_list = [[],cross_dir,knight_dir,lateral_dir,all_dir,all_dir]
slider     = [False,True,True,True,True,False]
x_algebraic = {'8': 0,'7': 1,'6': 2,'5': 3,'4': 4,'3': 5,'2': 6,'1': 7}
y_algebraic = {'a': 0,'b': 1,'c': 2,'d': 3,'e': 4,'f': 5,'g': 6,'h': 7}

In [58]:
#responisble for modifying chess board, uses eval_pseudo_legal_move() to if move is legal
def algebraic_to_arr_indices(s):
    y_1,x_1,y_2,x_2 = s
    x_1_alg = x_algebraic[x_1]
    y_1_alg = y_algebraic[y_1]
    x_2_alg = x_algebraic[x_2]
    y_2_alg = y_algebraic[y_2]
    move = [x_1_alg,y_1_alg,x_2_alg,y_2_alg]
    return move

def make_move(board,move,rights,player,king_pos,move_format="alg"):
    if move_format == "alg":
        move = algebraic_to_arr_indices(move)
    x_1_alg,y_1_alg,x_2_alg,y_2_alg = move
    cur_board = copy.copy(board[-1])
    cur_rights = copy.copy(rights[-1])
    cur_king_pos = copy.copy(king_pos[-1])

    legal_move,status,rights_new = eval_legal_move(board,move,rights,player,king_pos)
    
    if legal_move:
        one_hot_piece = cur_board[x_1_alg,y_1_alg]
        index = next(i for i,v in enumerate(one_hot_piece) if v!=0)
        piece = pieces[index]
        if piece == "Pawn":
            cur_board = move_pawn(cur_board,move,status)
        elif piece == "King":
            cur_board = move_king(cur_board,move,status)
            
            cur_king_pos.remove([x_1_alg,y_1_alg])
            cur_king_pos.append([x_2_alg,y_2_alg])
        else:
            cur_board[x_2_alg,y_2_alg] = cur_board[x_1_alg,y_1_alg]
            cur_board[x_1_alg,y_1_alg] = [0,0,0,0,0,0]
            
        board.append(cur_board)
        rights.append(rights_new)
        king_pos.append(cur_king_pos)
    else:
        print("did note make move")
        return board,status,rights,player,king_pos
    return board,status,rights,-player,king_pos

def eval_legal_move(board,move,rights,player,king_pos):
    cur_board = copy.copy(board[-1])
    cur_rights = copy.copy(rights[-1])
    cur_king_pos = copy.copy(king_pos[-1])
    legal_move,status,rights_new = eval_pseudo_legal_move(cur_board,move,cur_rights,player)
    king_is_now_in_chess = is_king_now_in_chess(board,rights,king_pos,cur_board,cur_rights,cur_king_pos,player)
    retval = legal_move and not king_is_now_in_chess
    return retval,status,rights_new


def generate_all_legal_moves(board,rights,player,king_pos):
    all_squares = [[i,j] for i in range(8) for j in range(8)]
    all_moves = [i+j for i in all_squares for j in all_squares]
    all_legal_moves = []
    for i in all_moves:
        legal,_,_ = eval_legal_move(board,i,rights,player,king_pos)
        if legal:
            all_legal_moves.append(i)
    return all_legal_moves

def perft(board,rights,player,king_pos,depth):
  #  pdb.set_trace()
    if depth == 0:
        return 1
    nodes = 0
    moves = generate_all_legal_moves(board,rights,player,king_pos)
    
    for i in moves:
        board,status,rights,player,king_pos = make_move(board,i,rights,player,king_pos,move_format="int array")
        nodes = nodes + perft(board,rights,player,king_pos,depth-1)
        board,status,rights,player,king_pos = unmake_move(board,rights,player,king_pos)
        
    return nodes

#retrieves the previous state of the board
def unmake_move(board,rights,player,king_pos):
    #pdb.set_trace()
    if len(board) == 1:
        #don't change player if board hasn't changed
        return board,"n",rights,player,king_pos

    #IMRPOVE THIS BY only keeping track of indicies with rights, instead of stacking entire 2darray
    board.pop()
    rights.pop()
    king_pos.pop()
    return board,"y",rights,-player,king_pos

def eval_castling(board,move,direction,rights,player):
    #pdb.set_trace()
    x_1,y_1,x_2,y_2 = move
    x_d,y_d = direction
    rook_offset = -2 if y_2 == 2 else 1
    y_plus_offset_to_large = y_2+rook_offset > 7
    if not y_plus_offset_to_large:
        has_rights = rights[x_1,y_1] == player and rights[x_2,y_2+rook_offset] == player
    else:
        has_rights = False
    if not has_rights:
        return False,"doesn't have rights to castle",rights
    is_blocked = check_if_blocked(board,move,direction)
    if is_blocked:
        return False,"blocked doing castle",rights
    # if castling queen-side
    if y_2 == 2:
        sensitive_squares = [[x_1,y_1],[x_2,y_2],[x_1,2],[x_1,3]]
        status = "king_move,castling,queenside"
    # if castling king-side
    else:
        status = "king_move,castling,kingside"
        #y_s_1,y_s_2 = 5,6
        sensitive_squares = [[x_1,y_1],[x_2,y_2],[x_1,5]]
       
    #squares that must not be under attack    
    is_in_check = check_if_squares_attackable(board,player,sensitive_squares,rights)
    if is_in_check:
        return False,"Is in check, trying castle",rights
    rights[x_1,y_1] = 0
    rights[x_2,y_2+rook_offset] = 0
    return True,status,rights

def check_if_squares_attackable(board,player,sensitive_squares,rights):
    #IMPROVE THIS BY maintaining lists for player positions
    #pdb.set_trace()
    enemy_positions = []
    for i in range(8):
        for j in range(8):
            if sum(board[i,j]) == -player:
                enemy_positions.append([i,j])
    
    for i in enemy_positions:
        for j in sensitive_squares:
            move = i + j
            legal,status,_ = eval_pseudo_legal_move(board,move,rights,-player)
            if legal:
                return True
    return False
            
def check_if_blocked(board,move,dir_reduced):
    x_1,y_1,x_2,y_2 = move
    x_dir,y_dir = dir_reduced
    while x_1+x_dir != x_2 or y_1+y_dir != y_2:
        x_1,y_1 = x_1+x_dir,y_1+y_dir
        one_hot_piece = board[x_1,y_1]
        if sum(one_hot_piece) != 0:
            return True
    return False

def reduce_to_sign(val):
    if val == 0:
        return 0
    elif val > 0:
        return 1
    else:
        return -1

    

In [6]:
def special_moves(cur_board,status):
    if "castling" in status:
        cur_board = move_rook(cur_board,status)
    elif status == "en-passant":
        cur_board = delete_pawn_en_passant(cur_board,status)
    else:
        sys.exit("not special move")

def move_king(cur_board,move,status):
    x_1_alg,y_1_alg,x_2_alg,y_2_alg = move
    
    if "castling" in status:
        side = status.split(',')
        rook_offset_from = -2 if side[2] == 'queenside' else 1
        rook_offset_to = 1 if side[2] == 'queenside' else -1
        
        #moving the rook
        cur_board[x_2_alg,y_2_alg+rook_offset_to] = cur_board[x_2_alg,y_2_alg+rook_offset_from]
        cur_board[x_2_alg,y_2_alg+rook_offset_from] = [0,0,0,0,0,0]
        
    # move king no matter what
    cur_board[x_2_alg,y_2_alg] = cur_board[x_1_alg,y_1_alg]
    cur_board[x_1_alg,y_1_alg] = [0,0,0,0,0,0]
    return cur_board
        
    
def move_pawn(cur_board,move,status):
    x_1_alg,y_1_alg,x_2_alg,y_2_alg = move
    
    #remove enemy from enpassant
    if "en-passant" == status:
        cur_board[x_2_alg-player,y_2_alg] = [0,0,0,0,0,0]
    
    cur_board[x_2_alg,y_2_alg] = cur_board[x_1_alg,y_1_alg]
    cur_board[x_1_alg,y_1_alg] = [0,0,0,0,0,0]
    return cur_board

def is_king_now_in_chess(board,rights,king_pos,cur_board,cur_rights,cur_king_pos,player):
    king_index = 0 if player == 1 else 1
       
    if len(king_pos) > 1:
        king_in_chess_before = check_if_squares_attackable(board[-1],player,[king_pos[-1][king_index]],rights[-1])
    else:
        king_in_chess_before = False
    
    king_in_chess_now = check_if_squares_attackable(cur_board,player,[cur_king_pos[king_index]],cur_rights)
    if king_in_chess_now and not king_in_chess_before:
        return True
    else:
        return False



In [7]:
def eval_pseudo_legal_move(board,move,rights,player_mover):
    x_1,y_1,x_2,y_2 = move
    
    #check out of bounds
    if x_2 < 0 or x_2 > 7 or y_2 < 0 or y_2 > 7:
        return False,"out of bounds",rights
    if x_1 < 0 or x_1 > 7 or y_1 < 0 or y_1 > 7:
        return False,"out of bounds",rights
    
    #clear en-passant rights for player 2 turns ago:
    rank_remove_rights = 0
    if player_mover == 1:
        rank_remove_rights = 3
    else:
        rank_remove_rights = 4
    for i in range(8):
        if rights[rank_remove_rights,i] == player_mover:
            rights[rank_remove_rights,i] = 0
            
    one_hot_piece = board[x_1,y_1]
    player = sum(one_hot_piece)
    if player != player_mover:
        return False,"not your piece to move",rights
    one_hot_piece_2 = board[x_2,y_2]
    player_2 = sum(one_hot_piece_2)
    if player_2 == player_mover:
        return False,"friendly fire",rights
    if player == 0:
        return False,"no piece at start position for move",rights
    index = next(i for i,v in enumerate(one_hot_piece) if v!=0)
    piece = pieces[index]
    
    #calc difference between positions
    x_d,y_d = x_2-x_1,y_2-y_1
    
    #special case for knight
    if piece == "Knight":
        return eval_knight_move(board,[x_d,y_d],move,player,rights,index)

    #special case for pawn
    elif piece == "Pawn":
        return eval_pawn_move(board,move,[x_d,y_d],player,rights)
    
    #try to reduce direction to eg. [1,0]
    if x_d == 0 and y_d == 0:
        return False,"move doesn't move piece",rights
    #if x_d and y_d don't have same absolute size and neither of them is 0, it's not a line
    if (abs(x_d) != abs(y_d)) and not(x_d == 0 or y_d ==0):
        return False,"move is not a line",rights
    x_dir = reduce_to_sign(x_d)
    y_dir = reduce_to_sign(y_d)
    
    #check direction is allowed for piece, and that it only slides (move more than one square) if it is allowed
    if [x_dir,y_dir] in direction_list[index]:
        blocked = check_if_blocked(board,move,[x_dir,y_dir])
        if not blocked:
            is_slider = slider[index]
            if not is_slider:
                #only king satisfies constraints this deep into if-statements
                if abs(x_d) > 1 or abs(y_d) > 1:
                    if piece == "King":
                        return eval_castling(board,move,[x_dir,y_dir],rights,player)
                    return False,"Piece is not a slider, but tries to slide",rights
                #normal king move of one
                return True,"king_move",rights
            return True,"",rights
        return False,"piece is blocked",rights
    return False,"Unrecognized violation",rights

In [8]:
def eval_knight_move(board,direction,move,player,rights,index):
        x_1,y_1,x_2,y_2 = move
        if not direction in direction_list[index]:
            return False,"Not legal move for knight",rights
        elif sum(board[x_2,y_2]) == player:
            return False,"attacking own player",rights
        else:
            return True,"",rights

def eval_pawn_move(board,move,direction,player,rights):
    x_1,y_1,x_2,y_2 = move
    x_d,y_d = direction
    rights[x_1,y_1] = 0
    #check direction
    if x_d == 0:
        return False,"no vertical move",rights

    if reduce_to_sign(player) != reduce_to_sign(x_d):
        return False,"pawn is moving the wrong direction",rights
    #if going straight
    if y_d == 0:
        #if single step
        if abs(x_d) == 1:
            #if landing square is empty
            if sum(board[x_2,y_2]) == 0:
                return True,"pawn_move",rights
            else:
                return False,"pawn blocked doing straight move of 1",rights
        #if double step
        elif abs(x_d) == 2:       
            #if pawn is at start position
            if (player == 1 and x_1 == 1) or (player == -1 and x_1 == 6):
                #pdb.set_trace()
                rights[x_2,y_2] = player
                return True,"",rights
            else:
                return False,"pawn tried to make double move, when not at start position",rights
        elif abs(x_d) > 2:
            return False,"pawn made to large a move",rights
    #if going lateral
    elif abs(y_d) == 1 and abs(x_d) == 1:
        #if landing square has enemy
        if sum(board[x_2,y_2]) == -player:
            return True,"",rights
        elif sum(board[x_2,y_2]) == 0:
            #check en-passant
            #if at fifth rank
            if (x_1 == 4 and player == 1) or (x_1 == 3 and player == -1):
                one_hot_piece = board[x_2-player,y_2] 
                index = next(i for i,v in enumerate(one_hot_piece) if v!=0)
                piece = pieces[index]
                #pdb.set_trace()
                if piece == "Pawn" and rights[x_2-player,y_2] == -player:
                    return True, "pawn_move,en-passant",rights
                else:
                    return False,"en-passant fail, either not pawn or don't have rights",rights
            else:
                return False,"en-passant fail, not at fifth rank",rights
        else:
            return False,"made cross-move without attacking",rights
    else:
        return False,"pawn has too large horizontal movement",rights

In [9]:
def curate_pgn(path):
    f = open(path,"r")
    s = f.read()
    s = s.replace("\n"," ")
    s = s.replace("+","")
    s_list = s.split(' ')
    s_list
    list_remove = [s_list[i] for i in range(len(s_list)) if i%3 == 0]
    for i in list_remove:
        s_list.remove(i)
    s_list.remove("")
    return s_list

curated = curate_pgn("extracted/two.txt")

In [71]:
def perft_test(depth):
    b = create_board()
    r = create_rights()
    k = create_king_positions() 
    p = 1
#    start = time.time()
    val = perft(b,r,p,k,depth)
#    end = time.time() 
    return val,end

cProfile.run('perft_test(2)')

         40442974 function calls (40442554 primitive calls) in 61.415 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a9c85466becd>:2(create_board)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a9c85466becd>:3(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a9c85466becd>:36(create_rights)
        8    0.000    0.000    0.000    0.000 <ipython-input-3-a9c85466becd>:37(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a9c85466becd>:47(create_king_positions)
      420    0.003    0.000    0.311    0.001 <ipython-input-58-61d3b21e7094>:11(make_move)
   168756    9.300    0.000   59.143    0.000 <ipython-input-58-61d3b21e7094>:124(check_if_squares_attackable)
   169680    0.244    0.000    0.566    0.000 <ipython-input-58-61d3b21e7094>:141(check_if_blocked)
  3057264    1.009    0.000    1.009    0.

In [ ]:
print(perft_results)

In [56]:
def play_game_until_error(curated_pgn):
    b = create_board()
    r = create_rights()
    k = create_king_positions() 
    s = ""
    p = -1
    count = 0
    for move in curated_pgn:
        b,s,r,p,k= make_move(b,move,r,p,k)
        count += 1
        if not(s in legal_outputs):#== "castling,k" or s == "castling,q" or s == "" or s == "en-passant"):
            print(count,s)
            break
    success = (s in legal_outputs)# == "castling,k" or s == "castling,q" or s == "" or s == "en-passant")
    return count, move,success,s
play_game_until_error(curated)

(47, 'e2c3', True, '')

In [ ]:
def start_game(player,moves=[]):
    #pdb.set_trace()
    b = create_board()
    r = create_rights()
    k = create_king_positions()
    s = ""
    p = player 
    count = 0
    do = "m"
    i = 0
    prev = "m"
    watching = len(moves) != 0
    stop = False
    pp_board(b[-1])


    while not stop:
        do = input()
#        if do == "":
#            do = prev
#            
        if watching:
            if i == len(moves):
                stop = True
                
            if do == "m":
                b,s,r,p,k = make_move(b,moves[i],r,p,k)
                prev == "m"
                
                print(moves[i])
            
            elif do == "u":
                b,s,r,p,k = unmake_move(b,r,p,k)
                if s == "n":
                    continue
                i = i-2
                prev = "u"
                
                print(moves[i])
            elif do == "exit":
                stop = True
            else:
                print("wrong input")
                continue
        else:  
            if do == "u":
                b,s,r,p,k = unmake_move(b,r,k)
                if s == "n":
                    print("didn't unmake")
                    continue
                i = i-2
                continue
            
            else:
                b,s,r,p,k = make_move(b,do,r,p,k)
                if not(s in legal_outputs):
                    print("wrong input")
                    print(s)
                    continue

        clear_output()
        pp_board(b[-1])
        print(s)

        sys.stdout.flush()
        count += 1
        i = i + 1
    clear_output()
    pp_board(b[-1])
start_game(-1,curated)